In [ ]:
#|default_exp text_editor

# Text Editor

This module contains a compliant implementation of Anthropic's Text Editor Tool API. More info on how the text editor tool work can be found [here](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/text-editor-tool#implement-the-text-editor-tool).

In [ ]:
#| export
from pathlib import Path

In [ ]:
#| hide
from claudette.core import *
from toolslm.funccall import *

In [ ]:
#| exports
def view(path:str, rng:tuple[int,int]=None, nums:bool=False):
    'View directory or file contents with optional line range and numbers'
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists(): return f'Error: File not found: {p}'
        if p.is_dir(): return f'Directory contents of {p}:\n' + '\n'.join([str(f) for f in p.glob('**/*')
                                                                           if not f.name.startswith('.')])
        
        lines = p.read_text().splitlines()
        s,e = 1,len(lines)
        if rng:
            s,e = rng
            if not (1 <= s <= len(lines)): return f'Error: Invalid start line {s}'
            if e != -1 and not (s <= e <= len(lines)): return f'Error: Invalid end line {e}'
            lines = lines[s-1:None if e==-1 else e]
            
        return '\n'.join([f'{i+s-1:6d} │ {l}' for i,l in enumerate(lines,1)] if nums else lines)
    except Exception as e: return f'Error viewing file: {str(e)}'

In [ ]:
print(view('_quarto.yml', (1,10), nums=True))

     1 │ project:
     2 │   type: website
     3 │   resources: 
     4 │     - "*.txt"
     5 │   preview:
     6 │     port: 3000
     7 │     browser: false
     8 │ 
     9 │ format:
    10 │   html:


In [ ]:
#| exports
def create(path: str, file_text: str, overwrite:bool=False) -> str:
    'Creates a new file with the given content at the specified path'
    try:
        p = Path(path)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p} containing:\n{file_text}'
    except Exception as e: return f'Error creating file: {str(e)}'

In [ ]:
print(create('test.txt', 'Hello, world!'))
print(view('test.txt', nums=True))

Error: File already exists: test.txt
     1 │ Let's add a 
     2 │ Let's add a 
     3 │ Let's add a 
     4 │ Let's add a 
     5 │ Let's add a 
     6 │ Let's add a 
     7 │ Let's add a 
     8 │ Let's add a 
     9 │ Let's add a 
    10 │ Let's add a 
    11 │ Hello, world!


In [ ]:
#| exports
def insert(path: str, insert_line: int, new_str: str) -> str:
    'Insert new_str at specified line number'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text().splitlines()
        if not (0 <= insert_line <= len(content)): return f'Error: Invalid line number {insert_line}'
            
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error inserting text: {str(e)}'

In [ ]:
insert('test.txt', 0, 'Let\'s add a new line')
print(view('test.txt', nums=True))

     1 │ Let's add a new line
     2 │ Let's add a 
     3 │ Let's add a 
     4 │ Let's add a 
     5 │ Let's add a 
     6 │ Let's add a 
     7 │ Let's add a 
     8 │ Let's add a 
     9 │ Let's add a 
    10 │ Let's add a 
    11 │ Let's add a 
    12 │ Hello, world!


In [ ]:
#| exports
def str_replace(path: str, old_str: str, new_str: str) -> str:
    'Replace first occurrence of old_str with new_str in file'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text()
        count = content.count(old_str)
        
        if count == 0: return 'Error: Text not found in file'
        if count > 1: return f'Error: Multiple matches found ({count})'
            
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error replacing text: {str(e)}'

In [ ]:
str_replace('test.txt', 'new line', '')
print(view('test.txt', nums=True))

     1 │ Let's add a 
     2 │ Let's add a 
     3 │ Let's add a 
     4 │ Let's add a 
     5 │ Let's add a 
     6 │ Let's add a 
     7 │ Let's add a 
     8 │ Let's add a 
     9 │ Let's add a 
    10 │ Let's add a 
    11 │ Let's add a 
    12 │ Hello, world!


In [ ]:
#| exports
def str_replace_editor(**kwargs: dict[str, str]) -> str:
    'Doc'
    cmd = kwargs.pop('command', '')
    if cmd == 'view': return view(**kwargs)
    elif cmd == 'str_replace': return str_replace(**kwargs) 
    elif cmd == 'create': return create(**kwargs)
    elif cmd == 'insert': return insert(**kwargs)
    elif cmd == 'undo_edit': return 'Undo edit is not supported yet.'
    else: return f'Unknown command: {cmd}'

In [ ]:
#| export
text_editor_conf = {'sonnet':     {'type': 'text_editor_20250124',
                                   'name': 'str_replace_editor'},
                    'sonnet-3-5': {'type': 'text_editor_20241022',
                                   'name': 'str_replace_editor'}}

In [ ]:
ns = mk_ns(str_replace_editor) # function from claudette text_editor module
c = Chat(models[1], tools=[text_editor_conf['sonnet']])
c.toolloop('Please explain what my _quarto.yml does. Use your tools', ns=ns)

Based on the content of your `_quarto.yml` file, here's an explanation of what it does:

### Overview
This is a configuration file for Quarto, which is a scientific and technical publishing system. Your configuration sets up a website project with specific formatting and layout options.

### Key Components:

1. **Project Configuration**:
   - `type: website` - Defines this as a website project (as opposed to a book or other format)
   - `resources: - "*.txt"` - Includes all .txt files as resources for the website
   - `preview` settings configure the development server to run on port 3000 without automatically opening a browser

2. **Format Settings**:
   - `html` section configures how HTML output is generated:
     - Uses the "cosmo" theme
     - Applies custom styles from styles.css
     - Enables table of contents (`toc: true`)
     - Configures code blocks with background, left border (#31BAE9 blue color), and "arrow" highlighting style
     - Sets specific grid layout dimensions for sidebar, body, margins, and gutters
     - Preserves markdown files (`keep-md: true`)
   - Also supports CommonMark format as an alternative output option

3. **Website Features**:
   - Enables Twitter card and Open Graph metadata for better social media sharing
   - Adds GitHub issue creation functionality (`repo-actions: [issue]`)
   - Configures a navbar with primary background color and search functionality
   - Sets up a floating sidebar

4. **Metadata Files**:
   - Includes two external configuration files: `nbdev.yml` and `sidebar.yml`
   - These likely contain additional settings for nbdev integration (for notebook-driven development) and sidebar structure

This configuration creates a website with a clean, code-friendly layout that's optimized for technical content. The settings suggest this might be a documentation site or technical blog with a focus on displaying code examples effectively.

<details>

- id: `msg_01PSn8nv3CEv8m5zuvCgHTvg`
- content: `[{'citations': None, 'text': 'Based on the content of your `_quarto.yml` file, here\'s an explanation of what it does:\n\n### Overview\nThis is a configuration file for Quarto, which is a scientific and technical publishing system. Your configuration sets up a website project with specific formatting and layout options.\n\n### Key Components:\n\n1. **Project Configuration**:\n   - `type: website` - Defines this as a website project (as opposed to a book or other format)\n   - `resources: - "*.txt"` - Includes all .txt files as resources for the website\n   - `preview` settings configure the development server to run on port 3000 without automatically opening a browser\n\n2. **Format Settings**:\n   - `html` section configures how HTML output is generated:\n     - Uses the "cosmo" theme\n     - Applies custom styles from styles.css\n     - Enables table of contents (`toc: true`)\n     - Configures code blocks with background, left border (#31BAE9 blue color), and "arrow" highlighting style\n     - Sets specific grid layout dimensions for sidebar, body, margins, and gutters\n     - Preserves markdown files (`keep-md: true`)\n   - Also supports CommonMark format as an alternative output option\n\n3. **Website Features**:\n   - Enables Twitter card and Open Graph metadata for better social media sharing\n   - Adds GitHub issue creation functionality (`repo-actions: [issue]`)\n   - Configures a navbar with primary background color and search functionality\n   - Sets up a floating sidebar\n\n4. **Metadata Files**:\n   - Includes two external configuration files: `nbdev.yml` and `sidebar.yml`\n   - These likely contain additional settings for nbdev integration (for notebook-driven development) and sidebar structure\n\nThis configuration creates a website with a clean, code-friendly layout that\'s optimized for technical content. The settings suggest this might be a documentation site or technical blog with a focus on displaying code examples effectively.', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 2175, 'output_tokens': 446}`

</details>